## SQLAlchemy
1. SQLAlchemy is a Object to Relation Mapping (ORM) library to interact with database tables (relations) using Python Objects instead of raw SQL queries.
2. SQLAlchemy has two components: __Core__ and __ORM__.
3. It provides the SQL toolkit/functions to perform operations with the database.

#### Advantages:
1. ORM: The code can be more Pythonic. So, it is more readable and understandable.
2. More focus can be put on application development and its functionalities rather than writing complex SQL queries.
3. Flexibility: It still allows to write raw SQL queries.
4. Database Independence: Easily switch between different databases with minimal code changes.
5. Safety: Prevents SQL injection by safely handling SQL queries and paramaters.

#### Components
1. __Engine__: Establishes the connectivity with database systems.
2. __Session__: Execute the SQL statements and handle transactions.
3. __Model__: Python class mapped to database table/relation.
4. __Column__: Python class attribute mapped to table columns.

In [8]:
#pip install sqlalchemy
#pip install psycopg2

In [9]:
username = "postgres"
password = ""
host = "localhost"
port = "5432"
database_name = "python_learning"

In [21]:
from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.orm import declarative_base, sessionmaker

In [22]:
# Create an engine connected to the Postgres database
url = f"postgresql+psycopg2://{username}:{password}@{host}:{port}/{database_name}"
engine = create_engine(url)

# Base class for our classes definitions
Base = declarative_base()

In [23]:
# Define a User model
class User(Base):
    __tablename__ = 'users'
    id = Column(Integer, primary_key=True)
    name = Column(String(50))
    age = Column(Integer)

# Create the tables in the database
Base.metadata.create_all(engine)

In [24]:
# Create a configured "Session" class
Session = sessionmaker(bind=engine)
# Create a Session
session = Session()

### Create - INSERT

In [26]:
# Create a new user
new_user = User(name='John Doe', age=30)

# Add the new user to the session
session.add(new_user)

# Commit the session to write the user to the database
session.commit()

In [27]:
# Fetch Auto Generated ID
new_user.id

1

In [28]:
# Inserting multiple users
new_users = [User(name='Glenn Maxwell', age=34), User(name='Mike Stark', age=28), User(name='Paul Brandon', age=42)]
session.add_all(new_users)
session.commit()

In [34]:
for _ in range(len(new_users)):
    print(new_users[_].id, end=" ")

2 3 4 

In [19]:
session.close()

### Read - SELECT

In [35]:
# Query all users
users = session.query(User).all()
for user in users:
    print(f"ID: {user.id}, Name: {user.name}, Age: {user.age}")

ID: 1, Name: John Doe, Age: 30
ID: 2, Name: Glenn Maxwell, Age: 34
ID: 3, Name: Mike Stark, Age: 28
ID: 4, Name: Paul Brandon, Age: 42


In [36]:
# Query a user by name
user = session.query(User).filter_by(name='John Doe').first()
print(f"ID: {user.id}, Name: {user.name}, Age: {user.age}")

ID: 1, Name: John Doe, Age: 30


In [37]:
#Query by age
users = session.query(User).filter(User.age > 30).all()
for user in users:
    print(f"ID: {user.id}, Name: {user.name}, Age: {user.age}")

ID: 2, Name: Glenn Maxwell, Age: 34
ID: 4, Name: Paul Brandon, Age: 42


### Update

In [38]:
# Query the user we want to update
user = session.query(User).filter_by(id = 3).first()

# Update the user's age
user.name = "Mitchell Stark"
user.age = 31

# Commit the session to write the changes to the database
session.commit()

### Delete

In [39]:
# Query the user we want to delete
user = session.query(User).filter_by(id = 4).first()

# Delete the user
session.delete(user)

# Commit the session to delete the user from the database
session.commit()

In [42]:
session.close()

### Related Tables

In [40]:
from sqlalchemy import create_engine, Integer, String, DateTime, ForeignKey, Boolean
from sqlalchemy.orm import declarative_base, mapped_column, Mapped, relationship, sessionmaker
from typing import List, Optional
from datetime import datetime

# Create an engine connected to the MySQL database
url = f"postgresql+psycopg2://{username}:{password}@{host}:{port}/{database_name}"
engine = create_engine(url)

# Base class for our classes definitions
Base = declarative_base()

In [41]:
class Customer(Base):
    __tablename__ = "customer"
    custId:Mapped[int] = mapped_column(name="cust_id", type_=Integer, primary_key=True, autoincrement="auto")
    custName:Mapped[str] = mapped_column(name="cust_name", type_=String(50), nullable=False)
    custPhone:Mapped[Optional[str]] = mapped_column(name="cust_phone", type_=String(20))
    orders: Mapped[List["Order"]] = relationship(back_populates="customer", cascade="all, delete-orphan")

class Order(Base):
    __tablename__ = "order"
    orderId:Mapped[int] = mapped_column("order_id", Integer, primary_key = True, autoincrement = "auto")
    orderDate:Mapped[datetime] = mapped_column("order_date", DateTime, default=datetime.now().date())
    orderItem:Mapped[Optional[str]] = mapped_column("item", String)
    orderStatus:Mapped[bool] = mapped_column("order_status", Boolean, default = True)
    custId: Mapped[int] = mapped_column("cust_id", ForeignKey("customer.cust_id"), nullable=False)
    updatedAt:Mapped[Optional[datetime]] = mapped_column("updated_at", DateTime, default=datetime.now(), onupdate=datetime.now())
    customer: Mapped["Customer"] = relationship(back_populates="orders")

# Create the tables in the database
Base.metadata.create_all(engine)

In [43]:
Session = sessionmaker(bind=engine)
session = Session()

### Create

In [74]:
data = [
    Customer(custName="Customer A", custPhone="123-456-7890", orders=[
        Order(orderDate=datetime.now().date(), orderItem="Item 1", orderStatus=True),
        Order(orderItem="Item 2"),
        Order(orderItem="Item 3", orderStatus=False)
    ]),
    Customer(custName="Customer B", custPhone="234-567-8901", orders=[
        Order(orderItem="Item 4"),
        Order(orderItem="Item 5", orderStatus=False),
        Order(orderItem="Item 6")
    ]),
    Customer(custName="Customer C", custPhone="345-678-9012", orders=[
        Order(orderItem="Item 7"),
        Order(orderItem="Item 8")
    ]),
    Customer(custName="Customer D", custPhone="456-789-0123", orders=[
        Order(orderItem="Item 9"),
        Order(orderItem="Item 10", orderStatus=False),
        Order(orderItem="Item 11", orderStatus=False),
    ]),
    Customer(custName="Customer E", custPhone="567-890-1234")
]

In [75]:
session.add_all(data)
session.commit()

##### Creating Order Seperately and assigning to existing Customer

In [76]:
session.add(Order(orderItem="Item 12", custId=5))
session.commit()

In [78]:
#session.add(Order(orderItem="Item 13")) #This will fail as foreign key is not passed
#session.commit()

##### Create new Customer

In [85]:
session.add(Customer(custName="Customer F", custPhone="567-890-1234"))
session.commit()

##### Assign Order to above newly created Customer

In [86]:
session.add(Order(orderItem="Item 13", custId=7))
session.commit()

### Read

In [44]:
customers = session.query(Customer).all()

In [45]:
for customer in customers:
    print(f"custId: {customer.custId}, custName: {customer.custName}, custPhone: {customer.custPhone}")

custId: 1, custName: Customer A, custPhone: 123-456-7890
custId: 2, custName: Customer B, custPhone: 234-567-8901
custId: 3, custName: Customer C, custPhone: 345-678-9012
custId: 4, custName: Customer D, custPhone: 456-789-0123
custId: 5, custName: Customer E, custPhone: 567-890-1234
custId: 7, custName: Customer F, custPhone: 567-890-1234


In [118]:
orders = session.query(Order).filter(Order.orderStatus == True).all()

In [119]:
for order in orders:
    print(f"orderId: {order.orderId}, orderItem: {order.orderItem}, custId: {order.custId}")

orderId: 1, orderItem: Item 1, custId: 1
orderId: 2, orderItem: Item 2, custId: 1
orderId: 4, orderItem: Item 4, custId: 2
orderId: 6, orderItem: Item 6, custId: 2
orderId: 7, orderItem: Item 7, custId: 3
orderId: 8, orderItem: Item 8, custId: 3
orderId: 9, orderItem: Item 9, custId: 4
orderId: 12, orderItem: Item 12, custId: 5
orderId: 15, orderItem: Item 13, custId: 7


In [120]:
orders = session.query(Order).filter(Order.orderId > 10).all()
for order in orders:
    print(f"orderId: {order.orderId}, orderItem: {order.orderItem}, custId: {order.custId}")

orderId: 11, orderItem: Item 11, custId: 4
orderId: 12, orderItem: Item 12, custId: 5
orderId: 15, orderItem: Item 13, custId: 7


In [121]:
orders = session.query(Order).filter(Order.custId == 1).all()
for order in orders:
    print(f"orderId: {order.orderId}, orderItem: {order.orderItem}, custId: {order.custId}")

orderId: 1, orderItem: Item 1, custId: 1
orderId: 2, orderItem: Item 2, custId: 1
orderId: 3, orderItem: Item 3, custId: 1


### Using SQLAlchemy Core

In [5]:
from sqlalchemy import select

In [6]:
statement = (
    select(Order).where(Order.custId == 1)
)

customer_orders = session.scalars(statement).all()

In [7]:
for order in customer_orders:
    print(f"orderId: {order.orderId}, orderItem: {order.orderItem}, orderDate: {order.orderDate}")

orderId: 1, orderItem: Item 1, orderDate: 2024-05-30 00:00:00
orderId: 2, orderItem: Item 2, orderDate: 2024-05-30 00:00:00
orderId: 3, orderItem: Item 3, orderDate: 2024-05-30 00:00:00


In [181]:
orders = session.scalars(
    (select(Order).where(Order.orderStatus == False))
).all()

In [183]:
for order in orders:
    print(f"orderId: {order.orderId}, orderItem: {order.orderItem}, custId: {order.custId}")

orderId: 3, orderItem: Item 3, custId: 1
orderId: 5, orderItem: Item 5, custId: 2
orderId: 10, orderItem: Item 10, custId: 4
orderId: 11, orderItem: Item 11, custId: 4


In [185]:
orders = session.scalars(
    (select(Customer).join(Order.customer, Customer.custId == Order.custId).where(Order.orderStatus == False))
).all()

In [186]:
for order in orders:
    print(order.__dict__)
    #print(f"orderId: {order.orderId}, orderItem: {order.orderItem}, custId: {order.custId}, custName: {order.custName}")

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x000001C9EBEAD660>, 'custPhone': '123-456-7890', 'custId': 1, 'custName': 'Customer A'}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x000001C9EBF23820>, 'custPhone': '234-567-8901', 'custId': 2, 'custName': 'Customer B'}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x000001C9EBF23880>, 'custPhone': '456-789-0123', 'custId': 4, 'custName': 'Customer D'}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x000001C9EBF23880>, 'custPhone': '456-789-0123', 'custId': 4, 'custName': 'Customer D'}


### Update

In [67]:
order = session.query(Order).filter(Order.orderId == 10).one_or_none()

In [68]:
order.orderStatus

False

In [69]:
order.orderStatus = True
session.commit()

In [70]:
order.orderStatus

True

### Delete

In [71]:
order = session.query(Order).filter(Order.orderId == 6).one_or_none()
order.orderItem

'Item 6'

In [72]:
session.delete(order)
session.commit()

In [74]:
order.orderId

6

#### Flush
1. Clear all the changes (SQL statements) from session's current transaction.
2. Session will be flushed automatically (autoflush) after session.commit() executed.

In [75]:
session.flush()

In [76]:
session.close()